# <center>  $\color{indigo}{\text{Data preparation }}$ </center>

## <center>  $\color{indigo}{\text{Bellevue University. }}$ </center>
## <center>  $\color{indigo}{\text{DSC 540 }}$ </center>
#### <center>  $\color{indigo}{\text{Movie Data Analysis}}$ </center>
### <center>  $\color{indigo}{\text{ Project Milestone - 4. }}$ </center>
### <center>  $\color{indigo}{\text{ SAMUEL ABOYE. }}$ </center>

# **Cleaning/Formatting Website Data**

#### **Overview**
After downloading the movie data from IMDb, we utilize these datasets to fetch additional information, clean and transform them for better integration, and enrich our existing dataset with detailed metadata. This process allows us to enhance our dataset with more comprehensive information, such as detailed ratings, cast and crew data, and other relevant movie information.

#### **Data Source Description**
The IMDb datasets provide a wide range of information about movies, TV shows, and their associated cast and crew. The data includes titles, ratings, crew details, and episode information. We plan to download various IMDb datasets and integrate them with our existing movie data to expand the dataset.

#### **IMDb Data Files Available for Download**
Documentation for these data files can be found on [IMDb Non-Commercial Datasets](https://datasets.imdbws.com/).

The data files available on this page are provided for non-commercial use only. As of March 18, 2024, the datasets on this page are backed by a new data source. There has been no change in location or schema, but if you encounter issues with the datasets following the March 18th update, please contact [imdb-data-interest@imdb.com](mailto:imdb-data-interest@imdb.com).

The datasets used for this purpose include:

- `name.basics.tsv.gz`
- `title.basics.tsv.gz`
- `title.crew.tsv.gz`
- `title.episode.tsv.gz`
- `title.ratings.tsv.gz`

These datasets were chosen for their comprehensive data offerings and reliability. However, it's important to note that these data files are provided for non-commercial use only, and there are legal implications for scraping the IMDb website directly.

#### **Workflow**

### **Step 1: Preparing the Data**
1. **Data Preparation**: 
    - Start by defining a function to download and read gzip files from URLs.
    - Load the IMDb datasets from their respective URLs.

In [1]:
import requests
import gzip
import pandas as pd
from io import BytesIO

In [2]:
# Function to download and read a gzip file from a URL
def download_and_read_gzip(url):
    response = requests.get(url)
    compressed_file = BytesIO(response.content)
    with gzip.open(compressed_file, 'rt', encoding='utf-8') as f:
        return pd.read_csv(f, delimiter='\t')

In [3]:
# URLs of the datasets
datasets_urls = {
    'title_basics': 'https://datasets.imdbws.com/title.basics.tsv.gz',
    'title_crew': 'https://datasets.imdbws.com/title.crew.tsv.gz',
    'title_episode': 'https://datasets.imdbws.com/title.episode.tsv.gz',
    'title_ratings': 'https://datasets.imdbws.com/title.ratings.tsv.gz',
    'name_basics': 'https://datasets.imdbws.com/name.basics.tsv.gz'
}


In [4]:
# Download and read each dataset into a dictionary of DataFrames
imdb_data = {}
for name, url in datasets_urls.items():
    print(f"Downloading and reading {name}...")
    imdb_data[name] = download_and_read_gzip(url)
    print(f"{name} loaded with {len(imdb_data[name])} records.")

C:\Users\samue\AppData\Local\Temp\ipykernel_18832\297354462.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f, delimiter='\t')


title_basics loaded with 10797940 records.
title_crew loaded with 10155966 records.
title_episode loaded with 8266814 records.
title_ratings loaded with 1440289 records.
name_basics loaded with 13520905 records.


In [7]:
print(imdb_data)

{'title_basics':              tconst  titletype               primarytitle  \
0         tt0000001      short                 carmencita   
1         tt0000002      short     le clown et ses chiens   
2         tt0000003      short             pauvre pierrot   
3         tt0000004      short                un bon bock   
4         tt0000005      short           blacksmith scene   
...             ...        ...                        ...   
10797935  tt9916848  tvepisode              episode #3.17   
10797936  tt9916850  tvepisode              episode #3.19   
10797937  tt9916852  tvepisode              episode #3.20   
10797938  tt9916856      short                   the wind   
10797939  tt9916880  tvepisode  horrid henry knows it all   

                      originaltitle isadult startyear endyear runtimeminutes  \
0                        carmencita     NaN      1894      \n              1   
1            le clown et ses chiens     NaN      1892      \n              5   
2         

### **Step 2: Data Cleaning and Transformation**
1. **Cleaning and Transforming Data**:
    - Apply several cleaning and transformation steps to ensure data consistency and readiness for integration.

In [9]:
# Apply transformations and cleaning steps to each dataset
for name, df in imdb_data.items():
    # Step 1: Replace Headers
    df.columns = [col.replace('.', '_').lower() for col in df.columns]
    
    # Step 2: Identify and Handle Missing Values
    missing_values = df.isnull().sum()
    # Fill missing values with appropriate default values
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna('Unknown')
        elif df[column].dtype in ['int64', 'float64']:
            df[column] = df[column].fillna(0)
    
    # Step 3: Fix Casing or Inconsistent Values
    # Ensure all text columns are in lowercase
    for column in df.select_dtypes(include='object').columns:
        df[column] = df[column].str.lower()
    
    # Step 4: Find and Remove Duplicates
    df = df.drop_duplicates()
    
    # Step 5: Format Data
    # Convert numerical columns to the appropriate type
    for column in df.columns:
        if df[column].dtype == 'object':
            try:
                df[column] = pd.to_numeric(df[column])
            except ValueError:
                continue
    
    # Save the cleaned dataset back to the dictionary
    imdb_data[name] = df

### **Step 3: Verify Data Structure for Integration**
1. **Verification**:
    - Conduct a basic metadata check to ensure that the cleaned datasets are properly structured for integration.

In [10]:
# Display a summary of the cleaned data
for name, df in imdb_data.items():
    print(f"\n{name} dataset:")
    print(df.info())
    print(df.head())


title_basics dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10797940 entries, 0 to 10797939
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titletype       object
 2   primarytitle    object
 3   originaltitle   object
 4   isadult         object
 5   startyear       object
 6   endyear         object
 7   runtimeminutes  object
 8   genres          object
dtypes: object(9)
memory usage: 741.4+ MB
None
      tconst titletype            primarytitle           originaltitle  \
0  tt0000001     short              carmencita              carmencita   
1  tt0000002     short  le clown et ses chiens  le clown et ses chiens   
2  tt0000003     short          pauvre pierrot          pauvre pierrot   
3  tt0000004     short             un bon bock             un bon bock   
4  tt0000005     short        blacksmith scene        blacksmith scene   

   isadult startyear endyear runtimeminutes                    

In [11]:
df.head()

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles
0,nm0000001,fred astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,lauren bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,brigitte bardot,1934,\n,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,john belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,ingmar bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [12]:
df.shape

(13520905, 6)

In [14]:
#save each cleaned dataset to a CSV file for further analysis
for name, df in imdb_data.items():
    df.to_csv(f'cleaned_{name}.csv', index=False)

### **Ethical Implications of Data Transformation**

When transforming and cleaning this movie dataset, several steps were taken to enhance data quality and usability, including standardizing headers and text formats, removing duplicates, and formatting date fields. Such processes are essential to making data analysis tasks more straightforward and ensuring accuracy.

#### Ethical Considerations
**Data Changes**

- **Textual Data and Formats**: We standardized textual data and reformatted dates to ensure uniformity and ease of analysis.
- **Duplicate Removal**: While removing duplicates prevents skewed statistical analyses, it's crucial to ensure that this does not inadvertently remove valid data that appears similar.

**Legal and Regulatory Guidelines**

The data files available from IMDb are provided for non-commercial use only. There are legal implications for scraping the IMDb website directly, and users should adhere to the terms of service provided by IMDb. General data handling and privacy principles should always be considered to maintain trust and integrity in data management.

**Transformation Risks**

- **Over-Cleaning**: There's a risk that over-cleaning might lead to the loss of critical data, particularly with automated processes that remove what appears to be duplicates or outliers without manual verification.

**Assumptions**

- **Duplicate Identifiers**: The assumption that duplicate IDs always indicate duplicate entries may not hold if there were errors in data collection or processing.

**Data Sourcing/Credibility**

- **Source Credibility**: The dataset was sourced from a public movie database, which is presumed to be credible. However, verification against multiple sources is recommended to enhance data reliability.

**Ethical Acquisition**

- **Public Data**: The dataset is assumed to be ethically sourced, containing publicly available information about movies without any personal data.

**Mitigation of Risks**

- **Transparency and Verification**: Maintain transparency about the data transformations and provide access to the raw data for verification purposes.
- **Best Practices**: Regularly update data handling practices to comply with emerging best practices and ethical standards.